##Dependencies and Imports:

In [0]:
'''
Dependencies:

- geopandas
- descartes
- geopy
- numba

'''

'''
Data files used in this notebook:

CAMS_ZIPCODE_STREET_SPECIFIC.cpg  
CAMS_ZIPCODE_STREET_SPECIFIC.shp
CAMS_ZIPCODE_STREET_SPECIFIC.dbf  
CAMS_ZIPCODE_STREET_SPECIFIC.shp.xml
CAMS_ZIPCODE_STREET_SPECIFIC.prj  
CAMS_ZIPCODE_STREET_SPECIFIC.shx
CAMS_ZIPCODE_STREET_SPECIFIC.sbn
CAMS_ZIPCODE_STREET_SPECIFIC.sbx

markets.csv (https://raw.githubusercontent.com/Nolanole/food-desert-DS/master/markets.csv)

'''
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
from functools import partial
import pyproj
from shapely.ops import transform

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

##Functions used for grid search for food deserts

In [0]:
def gen_points_inside_polygon(polygon, miles=0.5):
    '''Takes a polygon and number of miles as input, creates a square grid around the polygon edges
    and iterates in default one mile chunks exhaustively acorss the grid, returns list of all the points
    generated'''
    
    #convert miles to degrees latitude/longitude 
    lat_increment = miles * 0.0145
    lon_increment = miles * 0.02 #this is specifically for L.A. (varies based on lat)
    
    #get the four corners of a grid of max and min lat/long values 
    min_long, min_lat, max_long, max_lat = polygon.bounds
    
    #iterate over the grid one unit at a time, append lats and lons to lists:
    longs, lats = [], []
    
    #Latitudes: start one half unit away from min_lat:
    lat = min_lat + (lat_increment / 2)
    while lat < max_lat:
        lats.append(lat)
        lat += lat_increment
    lats.append(max_lat)
    
    #repeat for longitude; conversion is diff, 1 mile = 0.0145 degrees
    #start one half unit above the min_lat
    lon = min_long + (lon_increment / 2)
    while lon < max_long:
        longs.append(lon)
        #increase .02 degrees (longitude conversion for Los Angeles)
        lon += lon_increment
    longs.append(max_long)

    #iterate thru all latitude and longitude points, instantiate shapely Point objects
    #and append them to a list:
    points = []
    for i in range(len(longs)):
        for j in range(len(lats)):
            points.append(Point(longs[i], lats[j]))
            
    #iterate over the points and throw out ones outside the polygon:
    points_inside_polygon = []
    for i in range(len(points)):
        if polygon.contains(points[i]):
            points_inside_polygon.append(points[i])
            
    #if there are no points inside the polygon (very small polygon), add center as a point:    
    if len(points_inside_polygon) == 0:
        points_inside_polygon.append(polygon.centroid)
            
    #return the list of Shapely points which are bound inside the polygon        
    return points_inside_polygon

def make_circle(lon, lat, miles=1):
    km = miles * 1.60934
    proj_wgs84 = pyproj.Proj(init='epsg:4326')
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buff = Point(0, 0).buffer(km * 1000)
    return Polygon(transform(project, buff).exterior.coords[:])

def get_num_groceries_in_circle(df, circle):
    min_long, min_lat, max_long, max_lat = circle.bounds
    df = df[(df.LATITUDE >= min_lat)&(df.LATITUDE <= max_lat)&(df.LONGITUDE>=min_long)&(df.LONGITUDE<=max_long)]
    num_groceries = 0
    for i in range(len(df)):
        if circle.contains(df.iloc[i]['LOCATION']):
            num_groceries += 1
    return num_groceries
        
def search_for_deserts(zip_df, markets_df):
    #initialize master dict which will store dicts for each zip code:
    return_dict = {}
    
    #iterate over each row in zip code dataframe:
    for i in range(len(zip_df)):
        zip_code = zip_df.iloc[i].zip_code
        nearby_grocery_counts = []
        desert_list = []
        
        #iterate over all the test_points for each zip code:
        for point in zip_df.iloc[i].test_points:
            lon, lat = point.x, point.y
            #make the circle around the test_point:
            circle = make_circle(lon, lat)
            #get the num of groceries within the circle and add to list for that zip code:
            num_nearby = get_num_groceries_in_circle(markets_df, circle)
            nearby_grocery_counts.append(num_nearby)
            #if there are no nearby groceries, add that point to list of food deserts:
            if num_nearby == 0:
                desert_list.append(point)
        
        #record the num of food deserts found for each zip code        
        num_food_deserts = len(desert_list)
        
        #store all the zip_code data as dict and add to master dict:
        
        #some zip codes are broken into mulitple polygons: this makes sure the data for these zip codes are aggregated:
        if zip_code in return_dict:
            return_dict[zip_code]['nearby_grocery_counts'] += nearby_grocery_counts
            return_dict[zip_code]['desert_list'] += desert_list
            return_dict[zip_code]['num_food_deserts'] += num_food_deserts
        else:
            return_dict[zip_code] = {'nearby_grocery_counts': nearby_grocery_counts, 
                               'desert_list': desert_list, 'num_food_deserts': num_food_deserts}
        
    #finished all iteration, now return the master dict of dictionaries for each zip code:
    return return_dict


def unpack_desert_search(desert_search_results):
    #initialize dict w/ empty lists
    unpacked = {'zip_code': [], 'nearby_grocery_counts': [], 'desert_list': [], 'num_food_deserts': []}
    #first get the zip code:
    for k, v in desert_search_results.items():
        unpacked['zip_code'].append(k)
        #now get the other key, value pairs for that zip code:
        for key, val in v.items():
            unpacked[key].append(val)
    #convert the dict into pandas df:        
    unpacked_df = pd.DataFrame(unpacked)        
    
    #initialize empty list of deserts:
    desert_list = []
    #iterate over the dataframe desert_list column and extract each desert and append to list:
    for i in range(len(unpacked_df)):
        deserts = unpacked_df.iloc[i].desert_list
        if len(deserts) > 0:
            for desert in deserts:
                desert_list.append(desert)
    
    #drop the desert_list column from the df:
    unpacked_df = unpacked_df.drop(columns=['desert_list'])
    
    #iterate over the grocery number counts and convert it to avg nearby groceries for each zip code:
    def get_avg(num_list):
        return round(sum(num_list)/len(num_list),1)
    
    unpacked_df.nearby_grocery_counts = unpacked_df.nearby_grocery_counts.apply(get_avg)
    unpacked_df = unpacked_df.rename(columns={'nearby_grocery_counts': 'avg_num_nearby_groceries'})
    
    #return the zip_code dataframe, and the list of all food deserts found in grid search:
    return unpacked_df, desert_list

##Load and Clean the Markets Data and Zip code Polygons data

In [0]:
#load the data
#markets csv has been lightly pre-cleaned:
markets = pd.read_csv('https://raw.githubusercontent.com/Nolanole/food-desert-DS/master/markets.csv').drop(columns=['Unnamed: 0'])

#drop duplicates
markets = markets.drop_duplicates(subset='FACILITY ID', keep='first')

#drop one row w/ null location info:
markets = markets.dropna()

#rename some columns to match functions
markets = markets.rename(columns={'Location':'LOCATION','FACILITY LATITUDE': 'LATITUDE', 'FACILITY LONGITUDE': 'LONGITUDE'})

#filter out gas stations and other junk groceries
exclude_string = "smoke|fizz|news|ginseng|wealth|mochi|beauty|perfume|big lots|97 CENT|marshal|water|BOYS AND GIRLS CLUB|vitamin|beer|honeybaked|sporting|ross|cnn|office|carwash|airport|service station|LUIQUOR|almonds|farm|snack|sweet|cork|baby|muscle|yogurt|godiva|stub|diy|baskin|ice cream|liq|home depot|wine|spirit|eleven|car wash|staples|am pm|99 cent|best buy|forever 21|craft|7-11|duty|sugar|mobil|gift shop|rite aid|76|petrol|weight watchers|7 eleven|golden state|dollar tree|six flags|arco|dress for less|liquor|walgreens|edible|fuel|studios|jenny craig|macy's|candies|general nutrition center|wateria|shell|oil|7-eleven|99 cents|valero|chevron|24 HOUR FITNESS|98 CENTS|candy|party|am/pm|circle k|gnc|automotive|gas|cvs|pharmacy|daiso"
markets = markets[~markets['FACILITY NAME'].str.contains(exclude_string, flags=re.IGNORECASE, regex=True) == True]

#reset index:
markets = markets.reset_index(drop=True)

#convert Location col to Shapely Point:
for i in range(len(markets)):
    lat = markets.at[i, 'LATITUDE']
    lon = markets.at[i, 'LONGITUDE']
    markets.at[i, 'LOCATION'] = Point(lon, lat)
    
# Loading the zip code polygon files
polygon_file = 'CAMS_ZIPCODE_STREET_SPECIFIC.shp'
polygons = gpd.read_file(polygon_file)
polygons = polygons.to_crs(epsg=4326)

#new df w/ only the columns we need:
zip_df = polygons[['Zip_Num', 'geometry']].rename(columns={'Zip_Num':'zip_code'})

#call the function and make a new column in our zip_df that holds a lit of points inside the polygon
zip_df['test_points'] = zip_df.geometry.apply(gen_points_inside_polygon)

#create new column that displays the num of points inside the polygon
zip_df['num_points'] = zip_df['test_points'].apply(len)

In [0]:
markets.head()

,FACILITY ID,FACILITY NAME,PE DESCRIPTION,FACILITY ADDRESS,FACILITY CITY,FACILITY ZIP,LATITUDE,LONGITUDE,LOCATION
0,FA0023436,FOOD 4 LESS #775,"FOOD MKT RETAIL (2,000+ SF) LOW RISK",12222 E CARSON ST,HAWAIIAN GARDENS,90716,33.830948,-118.070507,POINT (-118.0705072947 33.8309480135)
1,FA0036549,TARGET STORES #,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",888 W ARROW HWY,SAN DIMAS,91773,34.103639,-117.823369,POINT (-117.8233687594 34.1036390196)
2,FA0052688,TOMAS MARKET,"FOOD MKT RETAIL (1-1,999 SF) HIGH RISK",4292 UNION PACIFIC AVE,LOS ANGELES,90023,34.016421,-118.180332,POINT (-118.1803317925 34.0164206789)
3,FA0167809,168 MARKET #806,"FOOD MKT RETAIL (1-1,999 SF) HIGH RISK",17120 COLIMA RD,HACIENDA HEIGHTS,91745,33.989837,-117.934159,POINT (-117.9341593463 33.9898366072)
4,FA0149302,7 DAYS MARKET,"FOOD MKT RETAIL (1-1,999 SF) LOW RISK",4309 GRIFFIN AVE,LOS ANGELES,90031,34.092305,-118.203969,POINT (-118.2039691739 34.0923046243)


##Perform the grid search, export the results:

In [0]:
full_search = search_for_deserts(zip_df, markets)
zip_code_groceries_df, all_deserts = unpack_desert_search(full_search)

In [0]:
#check how many food deserts were found:
len(all_deserts)

9757

In [0]:
#check the grocery density results dataframe:
zip_code_groceries_df.head()

,zip_code,avg_num_nearby_groceries,num_food_deserts
0,90001,75.8,0
1,90004,52.7,0
2,90006,105.8,0
3,90007,62.1,0
4,90008,7.9,2


In [0]:
#import os
#import pickle

#data = (zip_code_groceries_df, all_deserts)
#pickle.dump(data, open('grid_search_results_pickle', 'wb'))